In [2]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
sys.path.append('..')

import os
import json

import pandas as pd
import numpy as np
import math
from colormath.color_objects import XYZColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_objects import LabColor
from colormath.color_diff import delta_e_cie1976

In [20]:
datasets_path = os.path.join('..', 'datasets')

In [21]:
# Big color dataset
dpath_big = os.path.join(datasets_path, 'color-dict-big.json')
with open(dpath_big, 'r') as x:
    big = json.load(x)

In [22]:
# Standardize big color dataset
big_colors = pd.io.json.json_normalize(list(big['colors']))
big_colors = big_colors.drop(columns=['luminance'])
big_colors = big_colors.rename(columns={'rgb.r': 'r', 'rgb.g': 'g', 'rgb.b': 'b'})

In [23]:
# Set base colors
# Use the first 17 standard CSS1 colours
dpath_base = os.path.join(datasets_path, 'html-colors.json')
with open(dpath_base, 'r') as hc:
    base_col = json.load(hc)

In [27]:
def hex2cie(hexcolor):
    "Convert hex color to yuv"
    r = int(hexcolor[1:3], 16)
    g = int(hexcolor[3:5], 16)
    b = int(hexcolor[5:], 16)
    
    rgb = sRGBColor(r,g,b)
    lab = convert_color(rgb, LabColor, target_illuminant='d50')
    
    return lab

# Convert base_col rgb vals to yuv
base_col_cie = {}
for key, val in base_col.items():
    base_col_cie[key] = hex2cie(val)
    
# L1 distance between two YUV colors (not the best way)
# perhaps need to convert it to Lab* space before calculating similarity
def min_distance_colorname(x):
    min_delta = 3000000
    min_color = 'black'
    x_cie = hex2cie(x)
    for key, val_cie in base_col_cie.items():
        delta_e = delta_e_cie1976(x_cie, val_cie)
        if delta_e < min_delta:
            min_delta = delta_e
            min_color = key
            
    return min_color

# Try to get base color name based on default list
big_colors['basename'] = big_colors['hex'].apply(min_distance_colorname)
print(big_colors[0:18])

        hex                         name    b    g    r basename
0   #a59344           18th Century Green   68  147  165    olive
1   #7b463b               1975 Earth Red   59   70  123   maroon
2   #191970  20000 Leagues Under the Sea  112   25   25     navy
3   #d2d2c0                   400XT Film  192  210  210   silver
4   #9bafad             5-Masted Preußen  173  175  155   silver
5   #990066                8Bit Eggplant  102    0  153   purple
6   #3d1c02                    90% Cocoa    2   28   61   maroon
7   #d1edee     A Certain Shade Of Green  238  237  209    white
8   #d3dde4               A Dime a Dozen  228  221  211   silver
9   #456789   A Hint of Incremental Blue  137  103   69     gray
10  #bfaf92         A Pair of Brown Eyes  146  175  191   silver
11  #88ffcc              A State of Mint  204  255  136     aqua
12  #00b89f                   Aare River  159  184    0     teal
13  #05a3ad            Aare River Brienz  173  163    5     teal
14  #231f20              

In [ ]:
# Small color dataset
dpath_small = os.path.join(datasets_path, 'color-dict-small.json')
with open(dpath_small, 'r') as x:
    small = json.load(x)

In [ ]:
# Standardize small color dataset
small_colors = []
for colorPage in small['colorPage']:
    for entry in colorPage['colorEntry']:
        small_colors.append(entry)
small_colors = pd.io.json.json_normalize(list(small_colors))
small_colors = small_colors.rename(columns={
    'colorName': 'name', 'RGB8.red': 'r', 
    'RGB8.green': 'g', 'RGB8.blue': 'b'})
small_colors['name'] = small_colors['name'].apply(lambda n: re.sub('\(SW.*\)', '', n).strip())